<a href="https://colab.research.google.com/github/raja-jamwal/blog-agentic-architectures/blob/main/Part_3_Multi_Agent_Collaboration%2C_Memory%2C_and_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agents in Python and n8n in 2026
Companion to https://rajajamwal.substack.com/p/building-agents-in-python-and-n8n

Subscribe to my blog, https://rajajamwal.substack.com

## Multi-Agent Collaboration, Memory, and Learning

Welcome to Part 3 of the **Agentic Design Patterns** series.

In Parts 1 and 2, we built single agents that could reason and act. But in the real world, complex problems require **teams** and **context**. A doctor doesn't work without a nurse; a lawyer doesn't work without case files.

Today, we transform our isolated agents into collaborative, stateful systems.

We will cover:
1.  **Multi-Agent Collaboration:** Orchestrating specialized agents (Researcher + Writer).
2.  **Memory Management:** Enabling the agent to remember past interactions.
3.  **Learning (Few-Shot):** Teaching the agent to adapt its style using examples.

### The Stack
*   **Python**
*   **LangChain**
*   **OpenAI** (GPT-4o-mini)

### The n8n Connection
*   **Multi-Agent** = Chaining multiple **AI Agent Nodes**. For example, Agent A's output becomes Agent B's input variable.
*   **Memory** = Connecting a **Window Buffer Memory** node to your AI Agent. This automatically handles the context window.
*   **Learning** = Using a **Vector Store** to retrieve "successful past examples" and injecting them into the prompt (Few-Shot RAG).

In [1]:
# @title 1. Install Dependencies
!pip install -qU langchain langchain-openai langchain-core

import os
from getpass import getpass

# @title 2. Setup API Key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

# Initialize the Model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ Environment Setup Complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 10.1 MB/s eta 0:00:00
Enter your OpenAI API Key: ··········
✅ Environment Setup Complete.


## Pattern 7: Multi-Agent Collaboration (The Team)

**The Problem:** A "Jack of all trades" agent is often a master of none. If you ask one prompt to "Research deep quantum physics and write a poem about it," it might hallucinate the physics or write a boring poem.

**The Solution:** **Specialization**. Create one agent solely for Research (factual, dry) and another for Writing (creative, engaging). Pass the baton from one to the other.

**The Scenario:** We want to write a LinkedIn post about a technical topic.
1.  **Agent A (Researcher):** Extracts key facts.
2.  **Agent B (Writer):** Turns facts into a viral post.

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- Agent 1: The Researcher ---
# Role: Strict, factual, bullet points.
researcher_prompt = ChatPromptTemplate.from_template(
    """You are a Research Analyst.
    Given the topic '{topic}', list 3 key technical facts.
    Be concise and factual. No fluff."""
)
researcher_agent = researcher_prompt | llm | StrOutputParser()

# --- Agent 2: The Writer ---
# Role: Engaging, uses emojis, professional but fun.
writer_prompt = ChatPromptTemplate.from_template(
    """You are a LinkedIn Ghostwriter.
    Take the following facts and write a short, engaging post.
    Use emojis and a call to action.

    Facts:
    {facts}"""
)
writer_agent = writer_prompt | llm | StrOutputParser()

# --- Orchestration (The Handoff) ---
# We chain them together: Input -> Researcher -> Writer -> Output
chain = (
    {"facts": researcher_agent}
    | writer_agent
)

# --- Execution ---
topic = "The impact of AI on Junior Developers"
print(f"Topic: {topic}\n")
print("--- 🤖 Agents Collaborating... ---")
result = chain.invoke({"topic": topic})
print(result)

Topic: The impact of AI on Junior Developers

--- 🤖 Agents Collaborating... ---
🚀 **Unlocking Potential with AI in Development!** 💻✨

Hey, tech enthusiasts! 🌟 Did you know that AI is revolutionizing the way junior developers work? Here are three game-changing ways AI is making a difference:

1️⃣ **Code Generation Tools**: With tools like GitHub Copilot and OpenAI Codex, junior developers can receive real-time code suggestions. This means less time on boilerplate code and more time tackling complex problems! 🛠️

2️⃣ **Automated Testing & Debugging**: AI is stepping up the game in testing frameworks by predicting bugs and suggesting fixes. This not only enhances code quality but also streamlines the debugging process! 🐞🔍

3️⃣ **Skill Development**: AI-driven platforms are personalizing learning experiences, helping developers quickly master new languages and frameworks. Adaptive learning paths mean you can focus on what you need to grow! 📚🚀

The future of coding is here, and it’s powered

## Pattern 8: Memory Management (The Context)

**The Problem:** LLMs are stateless. If you say "Hi, I'm Raja" and then ask "What is my name?", the model will say "I don't know."

**The Solution:** **Conversation History**. We must store the back-and-forth messages and inject them into the prompt every time we talk to the model.

**The Scenario:** A simple chat bot that remembers your name and preferences.

In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder

# --- Define the Prompt with History ---
# MessagesPlaceholder is where the memory will be injected
memory_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = memory_prompt | llm | StrOutputParser()

# --- Setup the Memory Store ---
# In production, this would be a database (Redis, Postgres etc).
# Here, we use an in-memory dictionary for simplicity.
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# --- Wrap the Chain with History ---
conversation = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

# --- Execution ---
session_id = "user_123"

# Turn 1
print("🗣️ Turn 1:")
response1 = conversation.invoke(
    {"input": "Hi, my name is Raja. I like Python."},
    config={"configurable": {"session_id": session_id}}
)
print(response1)

# Turn 2 (The test)
print("\n🗣️ Turn 2:")
response2 = conversation.invoke(
    {"input": "What is my name and what language do I like?"},
    config={"configurable": {"session_id": session_id}}
)
print(response2)

🗣️ Turn 1:
Hi Raja! It's great to hear that you like Python. It's a versatile and powerful programming language. What do you enjoy most about Python? Are you working on any specific projects or learning something new?

🗣️ Turn 2:
Your name is Raja, and you like Python.


## Pattern 9: Learning (Few-Shot Adaptation)

**The Problem:** How do you get an agent to follow a very specific format or tone without writing a 10-page instruction manual?

**The Solution:** **Few-Shot Learning (In-Context Learning)**. Instead of *telling* the model what to do, you *show* it examples of what you want. The model "learns" the pattern from the context.

**The Scenario:** We want an agent that converts normal English into "Pirate Speak" but specifically in a JSON format.

In [4]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

# --- 1. Define Examples (The "Training" Data) ---
examples = [
    {"input": "Hello, how are you?", "output": "{\"text\": \"Ahoy matey! How be ye?\"}"},
    {"input": "Where is the bathroom?", "output": "{\"text\": \"Avast! Where be the head?\"}"},
    {"input": "I am hungry.", "output": "{\"text\": \"Me belly be rumblin' for grub!\"}"}
]

# --- 2. Create the Example Prompt ---
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# --- 3. Create the Few-Shot Prompt ---
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# --- 4. Assemble the Final Prompt ---
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a pirate. Output ONLY JSON."),
        few_shot_prompt, # Inject the examples here
        ("human", "{input}"),
    ]
)

# --- Execution ---
learning_chain = final_prompt | llm | StrOutputParser()

print("--- Testing Adaptation ---")
# We give a new input, and it should follow the JSON + Pirate pattern
result = learning_chain.invoke({"input": "The weather is nice today."})
print(result)

--- Testing Adaptation ---
{"text": "Aye, the sun be shinin' bright on the high seas!"}


## Summary

You have moved beyond basic scripts. Your agents now have:

1.  **Teamwork:** They can specialize and collaborate.
2.  **Memory:** They maintain context over time.
3.  **Adaptability:** They learn patterns from examples.

In **Part 4**, we will look at **Infrastructure**. We will explore the **Model Context Protocol (MCP)** and how to handle errors when things go wrong.